# OpenSSL Trial
This notebook serves to compare node-forge's AES with OpenSSL's AES implementation.

And maybe RSA, if we know how to test the implementation later on.

## Checking OpenSSL version
First we check and see if Colab's Linux environment has OpenSSL installed

In [1]:
!openssl version -a

OpenSSL 1.1.1  11 Sep 2018
built on: Mon Jul  4 11:25:51 2022 UTC
platform: debian-amd64
options:  bn(64,64) rc4(16x,int) des(int) blowfish(ptr) 
compiler: gcc -fPIC -pthread -m64 -Wa,--noexecstack -Wall -Wa,--noexecstack -g -O2 -fdebug-prefix-map=/build/openssl-wL7Fqk/openssl-1.1.1=. -fstack-protector-strong -Wformat -Werror=format-security -DOPENSSL_USE_NODELETE -DL_ENDIAN -DOPENSSL_PIC -DOPENSSL_CPUID_OBJ -DOPENSSL_IA32_SSE2 -DOPENSSL_BN_ASM_MONT -DOPENSSL_BN_ASM_MONT5 -DOPENSSL_BN_ASM_GF2m -DSHA1_ASM -DSHA256_ASM -DSHA512_ASM -DKECCAK1600_ASM -DRC4_ASM -DMD5_ASM -DAES_ASM -DVPAES_ASM -DBSAES_ASM -DGHASH_ASM -DECP_NISTZ256_ASM -DX25519_ASM -DPADLOCK_ASM -DPOLY1305_ASM -DNDEBUG -Wdate-time -D_FORTIFY_SOURCE=2
OPENSSLDIR: "/usr/lib/ssl"
ENGINESDIR: "/usr/lib/x86_64-linux-gnu/engines-1.1"
Seeding source: os-specific


Great! OpenSSL is present.
To conduct our tests we will need to store input and outputs in a text file.

## AES test

Test vectors for AES-128 from https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.197.pdf

We will use the CBC mode for AES-128; key and IV will be using the ones in the node-forge experiment.

Official example vector from AES paper

AES-128 example vector:  
Plaintext: 00112233445566778899aabbccddeeff  
Key: 000102030405060708090a0b0c0d0e0f  
IV(randomly generated by node-forge):  e9ed0e320d9ade4e77dbf6ad170af249

In [4]:
# reference from https://stackoverflow.com/questions/9889492/how-to-do-encryption-using-aes-in-openssl
# check last answer
# another good reference: https://stackoverflow.com/questions/16056135/how-to-use-openssl-to-encrypt-decrypt-files

!openssl enc -aes-128-cbc -in input.txt -out encrypted.txt -K 000102030405060708090a0b0c0d0e0f -iv e9ed0e320d9ade4e77dbf6ad170af249

To ensure this command works we run the encrypt command a second time. We will see the same encrypted bytes in encrypted2.txt

In [5]:
!openssl enc -aes-128-cbc -in input.txt -out encrypted2.txt -K 000102030405060708090a0b0c0d0e0f -iv e9ed0e320d9ade4e77dbf6ad170af249

In [6]:
!openssl enc -aes-128-cbc -in encrypted.txt -out output.txt -d -K 000102030405060708090a0b0c0d0e0f -iv e9ed0e320d9ade4e77dbf6ad170af249

Check output.txt, you will see the plaintext returned

Now we try using our own plaintext  
plain text: Hello World  
Key: 17a4daa1a71e1fd0f19f55b6e37f607c  
IV: 688f4c8b18a35dc068a5459535fcfc4a  

In [8]:
!openssl enc -aes-128-cbc -in input2.txt -out encrypted3.txt -K 17a4daa1a71e1fd0f19f55b6e37f607c -iv 688f4c8b18a35dc068a5459535fcfc4a

In [9]:
!openssl enc -aes-128-cbc -in encrypted3.txt -out output2.txt -d -K 17a4daa1a71e1fd0f19f55b6e37f607c -iv 688f4c8b18a35dc068a5459535fcfc4a